In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('anime.csv')

In [5]:
df.shape

(12294, 7)

In [7]:
df.size

86058

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [11]:
df = df.dropna()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12017 non-null  int64  
 1   name      12017 non-null  object 
 2   genre     12017 non-null  object 
 3   type      12017 non-null  object 
 4   episodes  12017 non-null  object 
 5   rating    12017 non-null  float64
 6   members   12017 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


In [15]:
df['description'] = df['name'] +' '+ df['genre'] + ' ' +df['type']+' episodes: '+ df['episodes']

In [17]:
df

,anime_id,name,genre,type,episodes,rating,members,description
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"Kimi no Na wa. Drama, Romance, School, Superna..."
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"Fullmetal Alchemist: Brotherhood Action, Adven..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"Gintama° Action, Comedy, Historical, Parody, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"Steins;Gate Sci-Fi, Thriller TV episodes: 24"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"Gintama&#039; Action, Comedy, Historical, Paro..."
...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,Toushindai My Lover: Minami tai Mecha-Minami H...
12290,5543,Under World,Hentai,OVA,1,4.28,183,Under World Hentai OVA episodes: 1
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,Violence Gekiga David no Hoshi Hentai OVA epis...
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,Violence Gekiga Shin David no Hoshi: Inma Dens...


In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F


In [21]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def get_embeddings(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

  return sentence_embeddings

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [23]:
sentences = ['Some great movie', 'Another funny movie']
result = get_embeddings(sentences)
print("Sentence embeddings:")
print(result)

Sentence embeddings:
tensor([[-1.0470e-01,  1.3669e-02, -3.7724e-02,  3.6088e-02, -3.0928e-02,
          3.3181e-02,  6.8035e-02, -4.1169e-02,  1.1926e-01, -4.1912e-02,
          3.9141e-02,  1.5677e-02, -3.6529e-02,  2.3321e-02, -3.2273e-02,
         -5.2346e-03,  3.8707e-02,  6.9842e-03, -2.7907e-02, -8.3663e-02,
         -5.1465e-02,  3.7350e-02,  3.9144e-02,  4.7031e-02, -1.6407e-01,
          2.1508e-02,  2.6223e-02,  1.4407e-03, -9.1594e-02, -3.3018e-02,
         -3.2289e-02,  6.4941e-02, -9.3422e-02,  3.0320e-02, -3.7254e-02,
          1.1184e-02,  3.4294e-02,  2.5122e-03, -2.7599e-02, -1.9730e-02,
         -3.3302e-02, -8.9150e-02,  9.6148e-02, -1.3116e-02,  3.5138e-02,
         -3.0958e-02, -2.3608e-02, -1.4274e-02,  4.8779e-02,  3.8363e-02,
         -2.9066e-02,  8.3276e-02, -5.3506e-02, -7.7774e-03,  4.9438e-02,
         -4.2659e-02, -5.9170e-02, -9.7170e-02,  3.5791e-02, -1.4385e-01,
          6.4405e-02, -3.7883e-02,  2.8620e-02, -5.5651e-02,  7.5088e-02,
          2.2148e

In [25]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [25]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

anime_embeddings = model.encode(df['description'].tolist())

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [26]:
from sklearn.metrics.pairwise import cosine_similarity



In [41]:
def get_recommendations(query, embeddings, df, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [43]:
query = "Funny anime I can watch with friends"
recommendations = get_recommendations(query, anime_embeddings, df)
print(recommendations[['name', 'genre']])

                                          name  \
7363  Sentou Yousei Shoujo Tasukete! Mave-chan   
8140            Anime TV de Hakken! Tamagotchi   
4294      SKET Dance: SD Character Flash Anime   
1061                        Isshuukan Friends.   
2850                       Oshiete! Galko-chan   

                                             genre  
7363  Comedy, Parody, Sci-Fi, Shounen, Super Power  
8140          Comedy, Fantasy, Kids, Slice of Life  
4294                       Comedy, School, Shounen  
1061        Comedy, School, Shounen, Slice of Life  
2850                 Comedy, School, Slice of Life  
